Begin by running the code cell below to set up code checking and the filepaths for the dataset.

In [1]:
# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.machine_learning.ex7 import *

# Set up filepaths
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv") 

In [2]:
# Import helpful libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np

# Load the data, and separate the target
iowa_file_path = '../input/train.csv'
home_data = pd.read_csv(iowa_file_path)
y = home_data.SalePrice

# Create X (After completing the exercise, you can return to modify this line!)

X = home_data.drop(columns=['Id', 'SalePrice'])
#X = X.drop(columns=['MSSubClass', 'LotFrontage', 'Street', 'Alley', 'Utilities', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
#'MasVnrType', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'Heating', 'Electrical', 'LowQualFinSF', 
#'BsmtFullBath', 'BsmtHalfBath', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 
#'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 
#'MiscVal', 'MoSold', 'YrSold', 'SaleType'])



#features = ['OverallQual', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea']
X.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [3]:
objects = list(X.select_dtypes(include=['object']).columns)
X_std = pd.get_dummies(X, columns=objects)
X.head()
features = list(X.columns)

In [4]:
# Select columns corresponding to features, and preview the data
#X = X.select_dtypes(exclude=['object'])
#X.head()

# Removing features which have std less than or equal to 1
#std= X.describe().loc['std', :]
#high_std_features = std[std>0.1].index
#X_std = X #[high_std_features]

#X.dtypes
#features = list(X_std.columns)
#features

In [5]:
#print(X_std.head())
X_std.dtypes
X_std.isna().sum()

MSSubClass                 0
LotFrontage              259
LotArea                    0
OverallQual                0
OverallCond                0
                        ... 
SaleCondition_AdjLand      0
SaleCondition_Alloca       0
SaleCondition_Family       0
SaleCondition_Normal       0
SaleCondition_Partial      0
Length: 288, dtype: int64

In [6]:
#X_std = X_std.drop(columns=['Id', 'LotFrontage', 'GarageYrBlt'] )
X_std = X_std.interpolate()
X_std.isna().sum()

features = list(X_std.columns)

In [7]:
import numpy as np
import matplotlib.pyplot as plt

#corr = X_std.corr().abs()
#mask = np.triu(np.ones_like(corr, dtype=bool))
#X_std_corr_tri = corr.mask(mask)

# Drop the columns with a correlation of 1
#to_drop = [c for c in X_std_corr_tri.columns if any(X_std_corr_tri[c] >= 1)]

#print(to_drop)
#X_std = X_std.drop(columns=to_drop)

In [8]:
#high_variance_features = X_std.columns[X_std.var() > 500]
#print(f"Features with variance greater than 1000: {high_variance_features}")

In [9]:
#for feature in high_variance_features:
 #   X_std[feature] = np.log(X_std[feature]+1)


In [10]:
to_remove = ['Exterior2nd_Other', 'Utilities_NoSeWa', 'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn', 'HouseStyle_2.5Fin', 
             'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'Exterior1st_ImStucc', 'Exterior1st_Stone', 
             'Heating_Floor', 'Heating_OthW', 'Electrical_Mix', 'GarageQual_Ex', 'PoolQC_Fa', 'MiscFeature_TenC']

features = [value for value in features if value not in to_remove]
X_std = X_std[features]

In [11]:
# Split into validation and training data
#t_X, v_X, t_y, v_y = train_test_split(X_std, y, test_size=0.3, random_state=1)
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X_std, y, test_size=0.3, random_state=1)

from sklearn.feature_selection import RFE
#best score n_features_to_select=20
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=288, step=50, verbose=1)
rfe.fit(train_X, train_y)


RFE(estimator=RandomForestRegressor(), n_features_to_select=288, step=50,
    verbose=1)

In [12]:
feat = X_std.columns[rfe.support_]
X_std = X_std[feat]
X_std.head()
features = list(X_std.columns)

In [13]:
#X_std.dtypes
#sns.pairplot(X_std)

In [14]:
# Split into validation and training data
#train_X, val_X, train_y, val_y = train_test_split(X_std, y, random_state=1)

# Scaling Data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_X_std = scaler.fit_transform(train_X)
val_X_std = scaler.transform(val_X)



In [15]:
# Define a random forest model
rf_model = RandomForestRegressor(n_estimators = 200, random_state=1)
#rf_model = xgb.XGBRegressor(random_state=1)
rf_model.fit(train_X_std, train_y)
rf_val_predictions = rf_model.predict(val_X_std)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))



Validation MAE for Random Forest Model: 17,255


# **Gradient Boosting MAE Score**

In [16]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for hyperparameter tuning
param_grid = {'n_estimators': [200, 250],
              'max_depth': [2, 3],
              'min_samples_split': [4, 6],
              'learning_rate': [0.1]}

# Create a GridSearchCV object for Gradient Boosting Regression
grid_search = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=5)

# Fit the GridSearchCV object to the training data
grid_search.fit(train_X_std, train_y)

# Get the best hyperparameters from the grid search
best_n_estimators = grid_search.best_params_['n_estimators']
best_max_depth = grid_search.best_params_['max_depth']
best_min_samples_split = grid_search.best_params_['min_samples_split']
best_learning_rate = grid_search.best_params_['learning_rate']

# Train a new Gradient Boosting Regression model with the best hyperparameters
gbr_model_best = GradientBoostingRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth,
                                            min_samples_split=best_min_samples_split, learning_rate=best_learning_rate, random_state=1)
gbr_model_best.fit(train_X_std, train_y)

# Predict the validation set using the best Gradient Boosting Regression model
gbr_val_predictions = gbr_model_best.predict(val_X_std)
gbr_val_mae = mean_absolute_error(gbr_val_predictions, val_y)

print("Validation MAE for Gradient Boosting Regression Model: {:,.0f}".format(gbr_val_mae))
print(grid_search.best_params_)


Validation MAE for Gradient Boosting Regression Model: 14,882
{'learning_rate': 0.1, 'max_depth': 3, 'min_samples_split': 6, 'n_estimators': 200}


In [17]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [250, 350],
    'max_depth': [4],
    'min_child_weight': [1],
    'learning_rate': [0.1]
}


# Create a GridSearchCV object for XGBoost Regression
grid_search = GridSearchCV(XGBRegressor(), param_grid, cv=5)

# Fit the GridSearchCV object to the training data
grid_search.fit(train_X_std, train_y)

# Get the best hyperparameters from the grid search
best_n_estimators = grid_search.best_params_['n_estimators']
best_max_depth = grid_search.best_params_['max_depth']
best_min_child_weight = grid_search.best_params_['min_child_weight']
best_learning_rate = grid_search.best_params_['learning_rate']

# Train a new XGBoost Regression model with the best hyperparameters
xgbr_model_best = XGBRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth,
                               min_child_weight=best_min_child_weight, learning_rate=best_learning_rate, random_state=1)
xgbr_model_best.fit(train_X_std, train_y)

# Predict the validation set using the best XGBoost Regression model
xgbr_val_predictions = xgbr_model_best.predict(val_X_std)
xgbr_val_mae = mean_absolute_error(xgbr_val_predictions, val_y)

print("Validation MAE for XGBoost Regression Model: {:,.0f}".format(xgbr_val_mae))
print(grid_search.best_params_)


Validation MAE for XGBoost Regression Model: 15,605
{'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 350}


In [18]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

# Train a new CatBoost Regression model with the best hyperparameters
cat_model_best = CatBoostRegressor(logging_level='Silent', random_state=1)
cat_model_best.fit(train_X_std, train_y)

# Predict the validation set using the best CatBoost Regression model
cat_val_predictions = cat_model_best.predict(val_X_std)
cat_val_mae = mean_absolute_error(cat_val_predictions, val_y)

print("Validation MAE for CatBoost Regression Model: {:,.0f}".format(cat_val_mae))



Validation MAE for CatBoost Regression Model: 14,512


In [19]:
# from sklearn.ensemble import StackingRegressor

# stacking_model = StackingRegressor(estimators = [('Gradient', gbr_model_best),
#                                                  ('Cat', cat_model_best)])

# stacking_model.fit(X_train, y_train)
# x_test_pred6 = stacking_model.predict(X_test)

# print("Stacking Model MAE on Testing Data is:",mean_absolute_error(y_test, x_test_pred6))

In [20]:
#import xgboost as xgb

# Split into validation and training data
#train_X, val_X, train_y, val_y = train_test_split(X_std, y, test_size = 0.3, random_state=1)

# Define a random forest model
#rf_model = RandomForestRegressor(n_estimators= 60, random_state=1)
#rf_model = xgb.XGBRegressor(random_state=1)
#rf_model.fit(train_X, train_y)
#rf_val_predictions = rf_model.predict(val_X)
#rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

#print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

In [21]:
#print(rf_model.feature_importances_)
#ask2 = rf_model.feature_importances_ > 0.01

#x_reduced = X_std.loc[:, mask2]
#print(x_reduced.columns)

The code cell above trains a Random Forest model on **`train_X`** and **`train_y`**.  

Use the code cell below to build a Random Forest model and train it on all of **`X`** and **`y`**.

In [22]:
#rf_model_on_full_data = RandomForestRegressor(n_estimators= 60, random_state=1)
model_on_full_data = CatBoostRegressor(logging_level='Silent', random_state=1)

# fit rf_model_on_full_data on all data from the training data
model_on_full_data.fit(X_std, y)

In [23]:



print(features)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'MSZoning_C (all)', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Grvl', 'Street_Pave', 'Alley_Grvl', 'Alley_Pave', 'LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_AllPub', 'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighb

Now, read the file of "test" data, and apply your model to make predictions.

In [24]:
# path to file you will use for predictions
test_data_path = '../input/test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)


In [25]:

test_data = pd.get_dummies(test_data, columns=objects)

# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features
# create test_X which comes from test_data but includes only the columns used for prediction.
# The list of columns is stored in a variable called 'features'
test_X = test_data[features]
test_X = test_X.interpolate()
#for feature in high_variance_features:
   # test_data[feature] = np.log(test_data[feature]+1)

In [26]:
# make predictions which we will submit. 
test_preds = model_on_full_data.predict(test_X)

Before submitting, run a check to make sure your `test_preds` have the right format.

# Generate a submission

Run the code cell below to generate a CSV file with your predictions that you can use to submit to the competition.

In [27]:
# Run the code to save predictions in the format used for competition scoring

output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)